# chunk

> internal utilities for chunkwise data processing

In [ ]:
#| default_exp chunk_

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import itertools

In [ ]:
#| export
def fill_slice(
    shape, # numpy arr, cupy arr, zarr,
    slices, # tuple of slice object, len == data_arr.ndim
):
    out_slices = []
    for i in range(len(slices)):
        slice_i = slices[i]
        if slice_i.start is None:
            start = 0
        else:
            start = slice_i.start
        assert start>=0
        assert start<shape[i]
        if slice_i.stop is None:
            stop = shape[i]
        else:
            stop = slice_i.stop
        assert stop > start
        assert stop <= shape[i]
        assert (slice_i.step is None) or (slice_i.step == 1)
        step = 1
        out_slices.append(slice(start,stop,step))
    return tuple(out_slices)

In [ ]:
#| export
def all_chunk_slices(
    shape, # np.array, cp.array,zarr
    chunks,
):
    '''get the slices for every input chunks'''
    out_slices = []
    for shape_, chunk_ in zip(shape,chunks):
        if chunk_ <0: chunk_ = shape_
        bound_1dim = np.arange(0,shape_+chunk_,chunk_)
        if bound_1dim[-1] > shape_: bound_1dim[-1] = shape_

        slice_1dim = []
        for j in range(bound_1dim.shape[0]-1):
            slice_1dim.append(slice(bound_1dim[j],bound_1dim[j+1]))
        out_slices.append(slice_1dim)
    out_slices = list(itertools.product(*out_slices))
    return out_slices

usage:

In [ ]:
a = np.zeros((50,100,11))
chunks = (15,40,6)
all_chunk_slices(a.shape,chunks)

[(slice(0, 15, None), slice(0, 40, None), slice(0, 6, None)),
 (slice(0, 15, None), slice(0, 40, None), slice(6, 11, None)),
 (slice(0, 15, None), slice(40, 80, None), slice(0, 6, None)),
 (slice(0, 15, None), slice(40, 80, None), slice(6, 11, None)),
 (slice(0, 15, None), slice(80, 100, None), slice(0, 6, None)),
 (slice(0, 15, None), slice(80, 100, None), slice(6, 11, None)),
 (slice(15, 30, None), slice(0, 40, None), slice(0, 6, None)),
 (slice(15, 30, None), slice(0, 40, None), slice(6, 11, None)),
 (slice(15, 30, None), slice(40, 80, None), slice(0, 6, None)),
 (slice(15, 30, None), slice(40, 80, None), slice(6, 11, None)),
 (slice(15, 30, None), slice(80, 100, None), slice(0, 6, None)),
 (slice(15, 30, None), slice(80, 100, None), slice(6, 11, None)),
 (slice(30, 45, None), slice(0, 40, None), slice(0, 6, None)),
 (slice(30, 45, None), slice(0, 40, None), slice(6, 11, None)),
 (slice(30, 45, None), slice(40, 80, None), slice(0, 6, None)),
 (slice(30, 45, None), slice(40, 80, None

In [ ]:
#| export
def all_chunk_slices_with_overlap(
    shape, 
    chunks, 
    depths,
):
    '''get the slices for every input chunks with overlap'''
    out_slices = []
    for shape_, chunk_, depth_ in zip(shape,chunks,depths):
        if chunk_ <0: chunk_ = shape_
        starts_1dim = np.arange(-depth_,shape_-depth_,chunk_)
        starts_1dim[starts_1dim<0] = 0
        ends_1dim = np.arange(chunk_+depth_,shape_+chunk_+depth_,chunk_)
        ends_1dim[ends_1dim>shape_] = shape_

        slice_1dim = []
        for j in range(starts_1dim.shape[0]):
            slice_1dim.append(slice(starts_1dim[j],ends_1dim[j]))
        out_slices.append(slice_1dim)
    out_slices = list(itertools.product(*out_slices))
    return out_slices

In [ ]:
a = np.zeros((50,100,11))
chunks = (15,40,6)
depth = [1,2,2] # or {0:1, 1:2, 2:3}
all_chunk_slices_with_overlap(a.shape,chunks,depth)

[(slice(0, 16, None), slice(0, 42, None), slice(0, 8, None)),
 (slice(0, 16, None), slice(0, 42, None), slice(4, 11, None)),
 (slice(0, 16, None), slice(38, 82, None), slice(0, 8, None)),
 (slice(0, 16, None), slice(38, 82, None), slice(4, 11, None)),
 (slice(0, 16, None), slice(78, 100, None), slice(0, 8, None)),
 (slice(0, 16, None), slice(78, 100, None), slice(4, 11, None)),
 (slice(14, 31, None), slice(0, 42, None), slice(0, 8, None)),
 (slice(14, 31, None), slice(0, 42, None), slice(4, 11, None)),
 (slice(14, 31, None), slice(38, 82, None), slice(0, 8, None)),
 (slice(14, 31, None), slice(38, 82, None), slice(4, 11, None)),
 (slice(14, 31, None), slice(78, 100, None), slice(0, 8, None)),
 (slice(14, 31, None), slice(78, 100, None), slice(4, 11, None)),
 (slice(29, 46, None), slice(0, 42, None), slice(0, 8, None)),
 (slice(29, 46, None), slice(0, 42, None), slice(4, 11, None)),
 (slice(29, 46, None), slice(38, 82, None), slice(0, 8, None)),
 (slice(29, 46, None), slice(38, 82, None

In [ ]:
#| export
def chunkwise_slicing_mapping(
    shape,
    chunks,
    depths,
):
    '''get the slices for every input chunks with overlap
    output chunks without overlap and their mapping slices'''
    in_slices = []; out_slices = []; map_slices = []
    for shape_, chunk_, depth_ in zip(shape,chunks,depths):
        if chunk_ <0: chunk_ = shape_
        in_starts_1dim = np.arange(-depth_,shape_-depth_,chunk_)
        in_starts_1dim[in_starts_1dim<0] = 0
        in_ends_1dim = np.arange(chunk_+depth_,shape_+chunk_+depth_,chunk_)
        in_ends_1dim[in_ends_1dim>shape_] = shape_
        out_starts_1dim = np.arange(0,shape_,chunk_)
        out_ends_1dim = np.arange(chunk_,shape_+chunk_,chunk_)
        out_ends_1dim[out_ends_1dim>shape_] = shape_

        assert len(in_starts_1dim) == len(out_starts_1dim)
        assert len(in_ends_1dim) == len(out_ends_1dim)

        in_slice_1dim = []; out_slice_1dim = []; map_slice_1dim = []
        for in_start, in_end, out_start, out_end in zip(in_starts_1dim, in_ends_1dim, out_starts_1dim, out_ends_1dim):
            in_slice_1dim.append(slice(in_start,in_end))
            out_slice_1dim.append(slice(out_start,out_end))
            offset = out_start-in_start
            map_slice_1dim.append(slice(offset,offset+out_end-out_start))

        in_slices.append(in_slice_1dim)
        out_slices.append(out_slice_1dim)
        map_slices.append(map_slice_1dim)

    in_slices = list(itertools.product(*in_slices))
    out_slices = list(itertools.product(*out_slices))
    map_slices = list(itertools.product(*map_slices))
    return in_slices, out_slices, map_slices

In [ ]:
a = np.zeros((50,100))
chunks = (15,40)
depth = [1,2] # or {0:1, 1:2, 2:3}
chunkwise_slicing_mapping(a.shape,chunks,depth)

([(slice(0, 16, None), slice(0, 42, None)),
  (slice(0, 16, None), slice(38, 82, None)),
  (slice(0, 16, None), slice(78, 100, None)),
  (slice(14, 31, None), slice(0, 42, None)),
  (slice(14, 31, None), slice(38, 82, None)),
  (slice(14, 31, None), slice(78, 100, None)),
  (slice(29, 46, None), slice(0, 42, None)),
  (slice(29, 46, None), slice(38, 82, None)),
  (slice(29, 46, None), slice(78, 100, None)),
  (slice(44, 50, None), slice(0, 42, None)),
  (slice(44, 50, None), slice(38, 82, None)),
  (slice(44, 50, None), slice(78, 100, None))],
 [(slice(0, 15, None), slice(0, 40, None)),
  (slice(0, 15, None), slice(40, 80, None)),
  (slice(0, 15, None), slice(80, 100, None)),
  (slice(15, 30, None), slice(0, 40, None)),
  (slice(15, 30, None), slice(40, 80, None)),
  (slice(15, 30, None), slice(80, 100, None)),
  (slice(30, 45, None), slice(0, 40, None)),
  (slice(30, 45, None), slice(40, 80, None)),
  (slice(30, 45, None), slice(80, 100, None)),
  (slice(45, 50, None), slice(0, 40, No

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()